In [0]:
# インポート
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler

In [0]:
# データセットの用意
data_train = pd.read_csv('./input/train.csv')
data_train = data_train[['GrLivArea', 'YearBuilt', 'SalePrice']].astype('float')
#data_test = pd.read_csv('./input/test.csv')
#X_test_submission = data_test[['GrLivArea', 'YearBuilt']].values

# 標準化
scaler = StandardScaler()
scaler.fit(data_train)
data_train = scaler.transform(data_train)

# 説明・目的変数
X = data_train[:, :2]
y = data_train[:, 2:].ravel()

# train.csvを学習用（train）8割、検証用（val）2割に分割する
X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.8, test_size=0.2)

# 【問題1】ブレンディング
ブレンディングとは? 以下分類だがわかりやすかった。  
<br>
1111100000  正解文字列（Ground truth）

1110100000  強学習器 (90%)  ← 良いモデルの予測結果は90%だった  
ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー
<br>
1111100000  正解文字列（Ground truth）

1011110100  弱学習器 (70%) ：1をよく予測できる

1101000010  弱学習器　(70%) ：0をよく予測できる

0110101001  弱学習器  (60%) ：何もうまく予測できない

1111100000  弱学習器の投票平均 (100%)  ← 複数のビミョーなモデルたちの平均を取ってみると100%になった
  
これが一番単純なブレンディング  
とりあえずはこれを試してみる  
単一モデルのmseを列挙して比較

In [0]:
# 各モデルインスタンス格納用
clf_dict = {}

# 線形回帰
clf_dict['LinearRegression'] = LinearRegression()

# SVM
clf_dict['SVR']  = SVR(gamma='auto')

# 決定木
clf_dict['DecisionTreeRegressor']  = DecisionTreeRegressor()

for clf_key in clf_dict:
    # 学習
    clf_dict[clf_key].fit(X_train, y_train) 
    
    # 予測実施
    y_pred = clf_dict[clf_key].predict(X_test)

    # mse
    print('{}のmse : {}'.format(clf_key, mean_squared_error(y_test, y_pred)))

LinearRegressionのmse : 0.5907964071344487
SVRのmse : 0.4242916980793748
DecisionTreeRegressorのmse : 0.7988866786751622


In [0]:
# 初期化
y_pred = np.zeros((len(X_test)))
weights = np.array([0.2, 0.35, 0.45])

# blending
for i, clf_key in enumerate(clf_dict):
    # 学習
    clf_dict[clf_key].fit(X_train, y_train) 

    # 予測実施
    y_pred_tmp = clf_dict[clf_key].predict(X_test)
    y_pred += y_pred_tmp * weights[i]

# mse
print('blendingごのmse : {}'.format(mean_squared_error(y_test, y_pred)))

blendingごのmse : 0.47878260776888093


SVRよりちょい高いくらい  
全ての単一モデルより精度が上がる組み合わせを3つ探してみる

In [0]:
# 初期化
y_pred = np.zeros((len(X_test)))
weights = np.array([0, 0.6, 0.4])

# blending
for i, clf_key in enumerate(clf_dict):
    # 学習
    clf_dict[clf_key].fit(X_train, y_train) 

    # 予測実施
    y_pred_tmp = clf_dict[clf_key].predict(X_test)
    y_pred += y_pred_tmp * weights[i]

# mse
print('blendingごのmse : {}'.format(mean_squared_error(y_test, y_pred)))

blendingごのmse : 0.42306606556640686


In [0]:
# 初期化
y_pred = np.zeros((len(X_test)))
weights = np.array([0.15, 0.7, 0.15])

# blending
for i, clf_key in enumerate(clf_dict):
    # 学習
    clf_dict[clf_key].fit(X_train, y_train) 

    # 予測実施
    y_pred_tmp = clf_dict[clf_key].predict(X_test)
    y_pred += y_pred_tmp * weights[i]

# mse
print('blendingごのmse : {}'.format(mean_squared_error(y_test, y_pred)))

blendingごのmse : 0.3824964842842491


重みを変えるだけでも結構下回る  
モデルのパラメータもいじってみる

In [0]:
# 初期化
y_pred = np.zeros((len(X_test)))
weights = np.array([0.15, 0.7, 0.15])

# 各モデルインスタンス格納用
clf_dict = {}

# 線形回帰
clf_dict['LinearRegression'] = LinearRegression()

# SVM
clf_dict['SVR']  = SVR(gamma='auto')

# 決定木
clf_dict['DecisionTreeRegressor']  = DecisionTreeRegressor(max_depth=5)

# blending
for i, clf_key in enumerate(clf_dict):
    # 学習
    clf_dict[clf_key].fit(X_train, y_train) 

    # 予測実施
    y_pred_tmp = clf_dict[clf_key].predict(X_test)
    y_pred += y_pred_tmp * weights[i]

# mse
print('blendingごのmse : {}'.format(mean_squared_error(y_test, y_pred)))

blendingごのmse : 0.3721107048406115


DecisionTreeの深さを5に指定したらより下がった。

# 【問題2】バギング
bootstrap aggregatingの略。
各学習器に使う学習用データをブートストラップサンプリングによって得て、  
その学習した学習器を予測に用いて最後アンサンブルするという方法。らしい  
決定木をバギングして、ランダムフォレストのように使ったりする方法もあるよう  
とりあえずやってみる

In [0]:
# 決定木
dtc = DecisionTreeRegressor()

# 初期化
y_pred = np.zeros((len(X_test)))
count = 0

for i in range(5):
    # ブートストラップサンプリング
    X_train, _, y_train, _ = train_test_split(X, y,train_size=0.8, test_size=0.2, shuffle=True)
    
    # 学習
    dtc.fit(X_train, y_train) 
    
    # 予測実施
    y_pred += dtc.predict(X_test)

    print('{}回目のmse : {}'.format(i+1, mean_squared_error(y_test, dtc.predict(X_test))))
    
    count += 1
    
y_pred = y_pred / count

# mse
print('bagginごのmse : {}'.format(mean_squared_error(y_test, y_pred)))

1回目のmse : 0.10647491723431106
2回目のmse : 0.08949663042176824
3回目のmse : 0.2529806626715098
4回目のmse : 0.10667115829002781
5回目のmse : 0.13900803436879167
bagginごのmse : 0.03839169927040804


これはおkそう　他のブレンディングとスタッキングを重点的にやる

# 【問題3】スタッキング
名前の通り訓練データを積み上げていく
<br>
複数学習機で推定→推定結果を特徴量として学習し推定→...を繰り返していくらしい
<br>
とりあえずはK=4, M=3でやってみる 

In [0]:
# ステージ0
# 各モデルインスタンス格納用
clf_dict = {}

# SVM
clf_dict['SVR']  = SVR(gamma='auto')

# 決定木
clf_dict['DecisionTreeRegressor']  = DecisionTreeRegressor(max_depth=5)

# 決定木
clf_dict['LinearRegression']  = LinearRegression()

# なん分割するか
K = 2

kf = KFold(n_splits=K)

# ブレンド用初期化
blend = np.empty((int(len(X)* 1 / K), 0))

for clf_key in clf_dict:
    # 予測を初期化
    y_pred = np.empty((int(len(X)* 1 / K),))
    for train_index, test_index in kf.split(X, y):
        # 学習
        clf_dict[clf_key].fit(X[train_index], y[train_index])
        # ブレンド
        y_pred += clf_dict[clf_key].predict(X[test_index])
        
    blend = np.append(blend, (y_pred / K)[:, np.newaxis], axis=1)
    

print(blend.shape)

# ステージ1

# 標準化
scaler = StandardScaler()
scaler.fit(blend)
blend = scaler.transform(blend)

dtr = DecisionTreeRegressor()
dtr.fit(blend, y[test_index])

y_pred = dtr.predict(blend)

# mse
print('bagginごのmse : {} (DecisionTreeRegressor)'.format(mean_squared_error(y[test_index], y_pred)))

rd = Ridge()
rd.fit(blend, y[test_index])

y_pred = rd.predict(blend)

# mse
print('bagginごのmse : {} (Ridge)'.format(mean_squared_error(y[test_index], y_pred)))

X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.75, test_size=0.25)

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

# mse
print('単一モデルのmse : {} (LinearRegression)'.format(mean_squared_error(y_test, y_pred)))

(730, 3)
bagginごのmse : 2.715056552079923e-10 (DecisionTreeRegressor)
bagginごのmse : 0.6236051985813775 (Ridge)
単一モデルのmse : 0.43317471191614787 (LinearRegression)


めちゃくちゃ小さい数値になったが、最終ステージでDecisionTreeRegressorを採用したところ単一モデルより低くなった